In [176]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [177]:
data = pd.read_csv("/Users/yiweihan/Desktop/FeedForward_Data_ellipse.csv")
X = data.values[:, 0:2] # Take only the first two features.
X = torch.tensor(X, dtype = torch.float)
y = data.values[:, 2]
y = torch.tensor(y, dtype = torch.long)

In [178]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 25)
        self.fc2 = nn.Linear(25, 25)
        self.fc3 = nn.Linear(25, 25)
        self.fc4 = nn.Linear(25, 25)
        self.fc5 = nn.Linear(25, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        #x = F.relu(self.fc6(x))
        x = self.fc5(x)
        return F.log_softmax(x) #return F.softmax(x)

In [179]:
def plot_decision_boundary(clf, X, y, filename):
    # Set min and max values and give it some padding
    #x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
    #y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
    x_min, x_max = -1.0, 1.0
    y_min, y_max = -1.0, 1.0
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    #Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    X_out = net(torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype = torch.float))
    Z = X_out.data.max(1)[1]
    # Z.shape
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral, s = 1)
    plt.savefig(filename)
    plt.close()

In [180]:
net = Net()
max_acc = 0
min_loss = 100
learning_rate = .0005
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
nepochs = 10000 #10000
data, target = X, y
for epoch in range(nepochs):
    optimizer.zero_grad()
    net_out = net(data)
    loss = criterion(net_out, target)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch ', epoch, 'Loss ', loss.item())
        net_out = net(data)
        pred = net_out.data.max(1)[1] 
        correctidx = pred.eq(target.data)
        ncorrect = correctidx.sum()
        accuracy = ncorrect.item()/len(data)
        if min_loss > loss:
            min_loss = loss
            min_loss_acc = accuracy
        if max_acc < accuracy:
            max_acc = accuracy
            max_acc_loss = loss
        print('Training accuracy is ', accuracy)
print('Best Training accuracy is ', max_acc, 'Loss ', max_acc_loss)
print('Training accuracy is ', min_loss_acc, 'Best Loss is ', min_loss)
        

<ipython-input-178-06c23a96d897>:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) #return F.softmax(x)


Epoch  0 Loss  0.6870610117912292
Training accuracy is  0.5565321914206296
Epoch  100 Loss  0.5206724405288696
Training accuracy is  0.8402073473067387
Epoch  200 Loss  0.2195829153060913
Training accuracy is  0.8973029824956803
Epoch  300 Loss  0.19788600504398346
Training accuracy is  0.8982044925249794
Epoch  400 Loss  0.19406528770923615
Training accuracy is  0.8988054992111787
Epoch  500 Loss  0.19194428622722626
Training accuracy is  0.8994065058973781
Epoch  600 Loss  0.19027382135391235
Training accuracy is  0.8997070092404778
Epoch  700 Loss  0.18884152173995972
Training accuracy is  0.9001577642551274
Epoch  800 Loss  0.18751178681850433
Training accuracy is  0.9002328900909022
Epoch  900 Loss  0.1860872060060501
Training accuracy is  0.9003831417624522
Epoch  1000 Loss  0.18482600152492523
Training accuracy is  0.9007587709413267
Epoch  1100 Loss  0.18320780992507935
Training accuracy is  0.9007587709413267
Epoch  1200 Loss  0.1816381812095642
Training accuracy is  0.9009090

In [181]:
plot_decision_boundary(net, X, y, 'Result2.1.pdf' )

<ipython-input-178-06c23a96d897>:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) #return F.softmax(x)


In [115]:
#The Accuracy is 0.9555255
#The loss is 0.1055
#The number of hidden layer is 4
#There are 25 neurons per layer
#The activation function is Relu